In [1]:
import pandas as pd
import numpy as np
from km_plusplus import KMeans
from SVD import Matrix
import matplotlib.pyplot as plt
import math

Loading Combined Data and the Genre Average Ratings Data

In [2]:
def load_data(file_path):
    return pd.read_csv(file_path)

In [3]:
file_path = "./EncodedCombined2.csv"  # Replace with your dataset file path
data = load_data(file_path)

avg_rating = load_data("./Genre_rating.csv")
# Assuming the data matrix is constructed with users as rows and movies as columns
dataMat = data# Exclude first column (user IDs)
# data_matrix.drop(columns=["Zip-code","Title"],inplace=True)
dataMat

,UserID,MovieID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Gender,Age,Occupation
0,1,1193,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,10
1,1,661,3,0,0,1,1,0,0,0,...,1,0,0,0,0,0,0,1,1,10
2,1,914,3,0,0,0,0,0,0,0,...,1,0,1,0,0,0,0,1,1,10
3,1,3408,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,10
4,1,2355,5,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,1,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
869365,4211,3791,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45,5
869366,4211,3806,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,45,5
869367,4211,3840,4,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,45,5
869368,4211,3766,2,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,45,5


Add New User Details Here

In [4]:
newUserID=7000
newUserMovies={
    25 : 4.0,
    54 : 5.0,
    65: 3.0,
    1200 : 4.0,
    3200 : 5.0,
    2980 : 1.0,
    99 : 2.0,
    455 : 2.0 
}

## Rating globally poor movies as poor
# newUserMovies={
#     3904 : 1.0,
#     3772 : 2.0,
#     3493: 1.0,
#     2217 : 1.0,
#     2039 : 2.0,
#     1528 : 1.0,
#     1386 : 1.0,
#     1142 : 2.0,
#     975 : 2.0 
# }

## Rating globally great movies as poor

# newUserMovies = {
#     3881 : 1.0,
#     3607 : 1.0,
#     3280 : 2.0,
#     3245 : 2.0,
#     1830 : 1.0,
#     787 : 2.0,
#     729 : 1.0,
#     53 : 1.0,
# }

newUserDetails = {
    'Age' : 18,
    'Gender' : 0,
    'Occupation' : 17
}


movieDetails=pd.read_csv("movies.csv")
movieDetails=movieDetails.set_index(keys='MovieID')
columns = ["UserID","MovieID","Rating","Action","Adventure","Animation","Children's","Comedy","Crime","Documentary","Drama","Fantasy","Film-Noir","Horror","Musical",
"Mystery","Romance","Sci-Fi","Thriller","War","Western" , "Gender", "Age", "Occupation"]

Rows=[]
for movieId,Rating in newUserMovies.items():
    newRow={}
    movie=movieDetails.loc[movieId]
    genres = movie['Genres'].split('|')
    newRow["UserID"]=newUserID
    newRow["MovieID"]=movieId
    newRow["Rating"]=Rating
    for genre in genres:
        newRow[genre]=int(1)
    for key, value in newUserDetails.items():
        newRow[key]=int(value)
    Rows.append(newRow)

newUserdf=pd.DataFrame(Rows,columns=columns)
newUserdf=newUserdf.fillna(int(0))

dataMat=pd.concat([dataMat,newUserdf])
dataMat=dataMat.drop_duplicates()
dataMat

,UserID,MovieID,Rating,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,Gender,Age,Occupation
0,1,1193,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10
1,1,661,3.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10
2,1,914,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1,1,10
3,1,3408,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10
4,1,2355,5.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3,7000,1200,4.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0,18,17
4,7000,3200,5.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,18,17
5,7000,2980,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,18,17
6,7000,99,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,18,17


In [5]:
dataMat["MovieID"]=dataMat["MovieID"].astype(np.int16)
dataMat["UserID"]=dataMat["UserID"].astype(np.int16)
for col in dataMat.columns:
    if(col!="MovieID" and col!="UserID"):
        dataMat[col]=dataMat[col].astype(np.int8)
print(dataMat.dtypes)

UserID         int16
MovieID        int16
Rating          int8
Action          int8
Adventure       int8
Animation       int8
Children's      int8
Comedy          int8
Crime           int8
Documentary     int8
Drama           int8
Fantasy         int8
Film-Noir       int8
Horror          int8
Musical         int8
Mystery         int8
Romance         int8
Sci-Fi          int8
Thriller        int8
War             int8
Western         int8
Gender          int8
Age             int8
Occupation      int8
dtype: object


Creating New Dataframe with average rating of each user for each genre

In [6]:
df=dataMat
# Initialize an empty dictionary to store genre-wise ratings for each user
genre_ratings = {}

# Iterate over each row in the DataFrame
for _, row in df.iterrows():
    user_id = row['UserID']
    rating = row['Rating']
    genres = ['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
              'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
    
    # Iterate over each genre and update the genre_ratings dictionary
    for genre in genres:
        if genre not in genre_ratings:
            genre_ratings[genre] = {}
        if user_id not in genre_ratings[genre]:
            genre_ratings[genre][user_id] = {'total_rating': 0, 'count': 0}
        if row[genre] == 1:  # Check if the movie belongs to the genre
            genre_ratings[genre][user_id]['total_rating'] += rating
            genre_ratings[genre][user_id]['count'] += 1

# Initialize an empty dictionary to store the final DataFrame values
data = {}

# Iterate over each user and calculate the average rating for each genre
for user_id in set(df['UserID']):
    user_data = {}
    for genre, values in genre_ratings.items():
        if user_id in values and values[user_id]['count']>0:
            user_data[genre] = values[user_id]['total_rating'] / values[user_id]['count']
        else:
            user_data[genre] = np.nan
    data[user_id] = user_data

# Create the final DataFrame
result_df = pd.DataFrame.from_dict(data, orient='index')


Adding user details before SVD

In [7]:
# age=df['Age']
# gender=df['Gender']
# Occupation=df['Occupation']
# result_df['Age']=age
# result_df['Gender']=gender
# result_df['Occupation']=Occupation

In [8]:
data_matrix=result_df
data_matrix=data_matrix.sort_index()
data_matrix

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,4.200000,4.000000,4.111111,4.250000,4.142857,4.000000,NaN,4.428571,4.00,NaN,NaN,4.285714,NaN,3.666667,4.333333,3.666667,5.000000,NaN
2,3.500000,3.736842,NaN,NaN,3.560000,3.583333,NaN,3.898734,3.00,4.000000,3.000000,NaN,3.333333,3.708333,3.588235,3.483871,3.733333,4.333333
3,3.956522,4.000000,4.000000,4.000000,3.766667,NaN,NaN,4.000000,4.50,NaN,2.666667,4.000000,3.000000,3.800000,3.833333,3.800000,4.000000,4.666667
4,4.157895,3.833333,NaN,4.000000,NaN,5.000000,NaN,4.166667,4.50,NaN,4.333333,NaN,NaN,4.000000,3.555556,3.500000,3.333333,4.500000
5,2.612903,3.000000,4.000000,3.833333,3.410714,3.285714,3.666667,3.096154,NaN,4.000000,2.800000,3.333333,3.125000,3.100000,3.066667,2.846154,3.500000,4.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3.000000,2.987952,3.911765,3.444444,3.203065,3.528302,3.909091,3.505376,3.00,4.058824,2.986486,3.709677,3.411765,3.352459,2.834320,3.142857,3.785714,3.642857
6037,3.642857,4.000000,4.000000,3.666667,3.576271,3.833333,4.000000,3.877551,4.25,3.444444,4.111111,4.000000,3.692308,3.681818,3.692308,3.705882,4.000000,3.750000
6038,3.000000,4.000000,3.666667,3.000000,3.833333,NaN,NaN,3.888889,NaN,NaN,2.500000,NaN,NaN,4.166667,4.000000,NaN,4.000000,NaN
6040,2.976190,2.818182,3.000000,4.000000,3.274510,3.920000,4.500000,3.821622,3.50,4.000000,2.590909,4.000000,4.454545,3.488889,3.473684,3.926829,3.695652,4.000000


Getting Set of Unique users that have rated a movie

In [9]:
user_ids = set(df['UserID'])
user_ids=sorted(user_ids)
user_ids

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 15,
 17,
 18,
 19,
 20,
 21,
 22,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 33,
 35,
 36,
 37,
 38,
 39,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 73,
 74,
 75,
 76,
 77,
 78,
 80,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 169,
 170,
 171,
 172,
 174,
 175,
 176,
 177,
 178,
 180,
 181,
 182,
 183,
 186,
 187,
 188,
 190,
 191,
 192,
 193,
 194,
 195,
 196,
 197,
 198,
 199,
 200,
 201,
 202,
 203,

In [10]:
def get_non_zero_eigenvalues(Sigma):
    # Extract diagonal elements of Sigma
    diagonal_elements = np.diag(Sigma)

    # Filter out values not close to zero
    non_zero_eigenvalues = [eigenvalue for eigenvalue in diagonal_elements if not math.isclose(eigenvalue, 0)]

    return non_zero_eigenvalues


Replacing with average value of user's rating

In [11]:
# A_mat = data_matrix.copy()
# A_watched=dataMat.pivot(index='UserID', columns='MovieID', values='Rating')
# row_means = A_mat.mean(axis=1, skipna=True)
# A_mat_new = A_mat
# for index, row in A_mat_new.iterrows():
#     mean=row_means.loc[index]
#     A_mat_new.loc[index] = row.fillna(mean)
# A_mat=A_mat_new
# A_mat

Filling with a genre's average rating

In [12]:
## Filling with a genre's average rating
avgdict = avg_rating.set_index('Genre')['Avg. Rating'].to_dict()
avg_dict = {key: float(value)  for key, value in avgdict.items() if value != 'Avg. Rating'}

# A = data_matrix.pivot(index='UserID', columns='Genre', values='Rating')
A_mat = data_matrix
A_watched=dataMat.pivot(index='UserID', columns='MovieID', values='Rating')
# print(avg_dict)
A_mat= A_mat.fillna(avg_dict)
A_np=A_mat.values
A_np

array([[4.2       , 4.        , 4.11111111, ..., 3.66666667, 5.        ,
        3.63777015],
       [3.5       , 3.73684211, 3.68486822, ..., 3.48387097, 3.73333333,
        4.33333333],
       [3.95652174, 4.        , 4.        , ..., 3.8       , 4.        ,
        4.66666667],
       ...,
       [3.        , 4.        , 3.66666667, ..., 3.57046605, 4.        ,
        3.63777015],
       [2.97619048, 2.81818182, 3.        , ..., 3.92682927, 3.69565217,
        4.        ],
       [4.        , 2.        , 3.68486822, ..., 4.        , 4.        ,
        3.63777015]])

In [13]:
A_mat=A_mat.round(2)
# print(A_mat.dtypes)
A_mat

,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
1,4.20,4.00,4.11,4.25,4.14,4.00,3.93,4.43,4.00,4.08,3.22,4.29,3.67,3.67,4.33,3.67,5.00,3.64
2,3.50,3.74,3.68,3.42,3.56,3.58,3.93,3.90,3.00,4.00,3.00,3.67,3.33,3.71,3.59,3.48,3.73,4.33
3,3.96,4.00,4.00,4.00,3.77,3.71,3.93,4.00,4.50,4.08,2.67,4.00,3.00,3.80,3.83,3.80,4.00,4.67
4,4.16,3.83,3.68,4.00,3.52,5.00,3.93,4.17,4.50,4.08,4.33,3.67,3.67,4.00,3.56,3.50,3.33,4.50
5,2.61,3.00,4.00,3.83,3.41,3.29,3.67,3.10,3.45,4.00,2.80,3.33,3.12,3.10,3.07,2.85,3.50,4.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,3.00,2.99,3.91,3.44,3.20,3.53,3.91,3.51,3.00,4.06,2.99,3.71,3.41,3.35,2.83,3.14,3.79,3.64
6037,3.64,4.00,4.00,3.67,3.58,3.83,4.00,3.88,4.25,3.44,4.11,4.00,3.69,3.68,3.69,3.71,4.00,3.75
6038,3.00,4.00,3.67,3.00,3.83,3.71,3.93,3.89,3.45,4.08,2.50,3.67,3.67,4.17,4.00,3.57,4.00,3.64
6040,2.98,2.82,3.00,4.00,3.27,3.92,4.50,3.82,3.50,4.00,2.59,4.00,4.45,3.49,3.47,3.93,3.70,4.00


Dataframe with ratings of every movie a user has watched

In [14]:
A_watched

MovieID,1,2,3,4,5,6,7,8,9,10,...,3942,3943,3944,3945,3946,3947,3948,3949,3950,3951
UserID,,,,,,,,,,,,,,,,,,,,,
1,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,NaN,NaN,NaN,2.0,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Computing SVD on the User and Genre Matrix

In [15]:
A = Matrix(A_mat)
U,Sigma,V = A.svd()

non_zero_diag = get_non_zero_eigenvalues(Sigma)
print("This is nonzero diag")
print(non_zero_diag)
U_red,Sigma_red,V_red = A.randomized_svd(len(non_zero_diag))  # Number of components for SVD 

These are evecs
[[ 2.31082038e-01 -6.56822819e-02 -2.54821468e-01  7.30240748e-02
   9.69729204e-02 -3.44318543e-02 -1.23568997e-01  1.77192344e-01
   4.25471954e-02  9.61032746e-03 -2.39377463e-01 -1.28894066e-02
  -2.90671620e-02 -1.33146013e-01 -1.37719607e-01 -1.21831674e-01
   8.28398275e-01 -1.29492352e-01]
 [ 2.29891794e-01  6.42141372e-02 -3.09619621e-01  6.92803359e-02
   1.87295889e-01 -3.39254989e-02 -8.27959301e-02  5.46342327e-02
   1.14475177e-01  1.00844930e-01 -3.45049425e-01 -9.79102581e-02
   1.15972809e-01 -6.63308417e-01 -5.18165362e-02  1.58292967e-01
  -4.03982826e-01 -4.43441558e-04]
 [ 2.35733479e-01  4.32439866e-01  2.52590917e-01 -1.82524782e-01
  -1.22338329e-01  9.10807342e-02  2.01170878e-01  4.30488451e-01
   1.61344857e-01 -1.91380332e-02 -4.18409161e-02  5.63855588e-01
   2.47337122e-01 -3.90342649e-02 -3.66941689e-02 -3.29853569e-02
  -2.48036402e-03 -5.78259244e-03]
 [ 2.24950540e-01  5.61774021e-01  2.98888364e-02 -1.11375001e-01
  -7.37213681e-02  2.

Saving SVD results

In [16]:
np.savetxt('U_redFinal1.txt', U_red)
np.savetxt('Sigma_redFinal1.txt', Sigma_red)
np.savetxt('V_redFinal1.txt', V_red)

In [17]:
U_red = np.loadtxt('U_redFinal1.txt')
Sigma_red = np.loadtxt('Sigma_redFinal1.txt')
V_red = np.loadtxt('V_redFinal1.txt')
print(U_red.shape)
print(Sigma_red.shape)
print(V_red.shape)
# Reshape Sigma_red to a diagonal matrix
# Sigma_red = np.diag(Sigma_red)

(5330, 18)
(18, 18)
(18, 18)


Elbow method code to determine optimum k value

In [18]:
def elbow_method(data, max_clusters=4 , skip_factor=1):
    distortions = []
    # Generate x-axis values with skip_factor increments
    x_values = range(skip_factor, max_clusters + 1, skip_factor)
    for i in x_values:
        km = KMeans(i)
        km.fit(data)
        distortions.append(km.get_cost())  # km.inertia_ gives the distortion (cost)
    # Plotting the elbow curve
    plt.plot(x_values, distortions, marker='o')
    plt.xlabel('Number of clusters')
    plt.ylabel('Distortion')
    plt.title('Elbow Method')

# Assuming your data is in V_red_T
# Perform Elbow Method
new_mat = U_red
new_mat

array([[ 0.01489685,  0.01247067, -0.00641673, ...,  0.01469067,
         0.00709215, -0.00794414],
       [ 0.01336692,  0.00012688,  0.00267143, ...,  0.0014712 ,
        -0.00902856, -0.00851495],
       [ 0.0142924 ,  0.0152622 , -0.00842005, ...,  0.00142839,
         0.00165873, -0.0027809 ],
       ...,
       [ 0.01351281, -0.00108883,  0.00320131, ...,  0.00514046,
        -0.03667429,  0.0084667 ],
       [ 0.01344377, -0.00290034,  0.01694497, ..., -0.01407928,
        -0.01303901,  0.00173797],
       [ 0.01309643,  0.00110181, -0.0077667 , ..., -0.00363693,
         0.05108462,  0.04683107]])

Generating Map of row in dataframe to UserID 

In [19]:
user_index = {}
i =0
for x in user_ids:
    user_index[int(x)] = i 
    i+=1

Adding User Details to the latent representation of Users obtained above

In [20]:
new_mat_df=pd.DataFrame(new_mat)
# print(dataMat)
totalData=dataMat.copy()
totalData=totalData.set_index(keys="UserID")
gendermap={}
agemap={}
occupationmap={}
for x in user_ids:
    gendermap[user_index[x]]=np.unique(totalData.loc[x]['Gender'])[0]
    agemap[user_index[x]]=np.unique(totalData.loc[x]['Age'])[0]
    occupationmap[user_index[x]]=np.unique(totalData.loc[x]['Occupation'])[0]

new_mat_df['Age']=agemap
new_mat_df['Gender']=gendermap
new_mat_df['Occupation']=occupationmap
new_mat=new_mat_df.values
new_mat

array([[ 1.48968495e-02,  1.24706725e-02, -6.41673182e-03, ...,
         1.00000000e+00,  1.00000000e+00,  1.00000000e+01],
       [ 1.33669199e-02,  1.26877705e-04,  2.67142734e-03, ...,
         5.60000000e+01,  0.00000000e+00,  1.60000000e+01],
       [ 1.42923980e-02,  1.52621989e-02, -8.42004643e-03, ...,
         2.50000000e+01,  0.00000000e+00,  1.50000000e+01],
       ...,
       [ 1.35128108e-02, -1.08883149e-03,  3.20131387e-03, ...,
         5.60000000e+01,  1.00000000e+00,  1.00000000e+00],
       [ 1.34437724e-02, -2.90034092e-03,  1.69449676e-02, ...,
         2.50000000e+01,  0.00000000e+00,  6.00000000e+00],
       [ 1.30964262e-02,  1.10180831e-03, -7.76669787e-03, ...,
         1.80000000e+01,  0.00000000e+00,  1.70000000e+01]])

In [21]:
new_mat_df

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,Age,Gender,Occupation
0,0.014897,0.012471,-0.006417,0.004205,0.007861,-0.004152,-0.009854,0.006058,0.007948,-0.002410,...,0.002793,-0.020341,0.008607,0.012534,0.014691,0.007092,-0.007944,1,1,10
1,0.013367,0.000127,0.002671,0.014919,-0.010960,-0.001943,0.000730,0.004745,0.003851,0.000645,...,-0.004540,0.006029,-0.001948,0.005021,0.001471,-0.009029,-0.008515,56,0,16
2,0.014292,0.015262,-0.008420,0.025473,0.000158,0.002075,0.010225,-0.004123,0.000736,-0.011197,...,0.000319,0.002942,0.004179,-0.009345,0.001428,0.001659,-0.002781,25,0,15
3,0.014611,-0.001171,-0.012952,-0.001117,-0.013279,0.007126,0.015612,-0.005773,-0.023884,-0.022995,...,-0.006586,0.001965,-0.019783,0.009883,-0.022263,0.013069,-0.022961,45,0,7
4,0.012334,0.013509,0.010381,0.007941,-0.010780,0.006524,0.021179,-0.001218,0.005429,-0.005142,...,-0.000151,-0.004735,0.000523,0.011615,0.001397,-0.012597,0.017503,25,0,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5325,0.012614,0.004541,0.016491,0.002726,-0.008889,-0.001917,0.004554,0.000932,0.004327,-0.003370,...,0.004036,-0.005712,-0.004877,0.000554,-0.006822,-0.003394,0.000627,25,1,15
5326,0.014100,0.007270,-0.012796,-0.007064,-0.006683,-0.007289,0.004386,-0.007294,-0.007405,0.001479,...,0.013210,-0.001830,-0.010384,-0.001067,-0.003481,-0.009040,-0.002073,45,1,1
5327,0.013513,-0.001089,0.003201,0.012847,0.011068,-0.001830,-0.006255,0.000859,0.004018,0.001190,...,0.003616,0.018609,0.005549,0.017321,0.005140,-0.036674,0.008467,56,1,1
5328,0.013444,-0.002900,0.016945,0.010234,0.001111,0.007501,0.003173,-0.015831,-0.021282,0.013507,...,-0.009692,-0.020154,0.020841,-0.000644,-0.014079,-0.013039,0.001738,25,0,6


Running Elbow Method

In [22]:
# elbow_method(new_mat,120,3)

Fitting Kmeans++ with the optimum k value

In [23]:
km = KMeans(40)
km.fit(new_mat)

Recommending Movies based on the UserID passed

In [24]:
# Recommend movies for a particular user
def recommend_movies(user_id, data, users_watched,user_idx,cluster_labels):
    user_cluster = cluster_labels[user_idx[user_id]]
    # print(user_cluster)
    cluser_users = []
    for row,cluster in cluster_labels.items():
        if cluster == user_cluster and row!=user_idx[user_id]:
            cluser_users.append(users_watched.iloc[row]) 

    rating_movie = {}
    movie_rating_count={}
    for df in cluser_users:
        for key,value in df[df.notnull()].items():
            if int(key) not in rating_movie:
                rating_movie[int(key)] = 0
                movie_rating_count[int(key)] = 0
            
            rating_movie[int(key)]+=float(value)
            movie_rating_count[int(key)]+=1

    for key,value in movie_rating_count.items():
        if(value>0):
            rating_movie[key]=rating_movie[key]/float(value)

    final_list = sorted(rating_movie.items(), key=lambda x: x[1],reverse=True)

    movies=pd.read_csv("movies.csv")
    movies=movies.set_index(keys=['MovieID'])
    recommendations=[]
    for x in final_list:
        movie=movies.loc[x[0]]
        recommendations.append([x[0],x[1],movie['Title'],movie['Genres']])

    return recommendations

In [25]:
cluster_labels = km.get_pointCluster()

print(f"Cluster Labels: {cluster_labels}")
print(f"User_Index : {user_index}")
# User ID for which we want to recommend movies
user_id = newUserID  # Replace with the desired user ID

# Recommend movies for the user
recommended_movies = recommend_movies(user_id, new_mat,A_watched,user_index,cluster_labels)
print("Recommended movies for User", user_id)
for movie in recommended_movies[:5]:  # Display top 10 recommended movies
    print(f"Title : {movie[2]} , MovieID : {movie[0]} , Rating : {movie[1]} , Genres : {movie[3]}")  # Assuming the first column is movie titles


Cluster Labels: {0: 11, 1: 19, 2: 24, 3: 8, 4: 0, 5: 15, 6: 3, 7: 24, 8: 38, 9: 3, 10: 13, 11: 24, 12: 21, 13: 28, 14: 16, 15: 6, 16: 11, 17: 24, 18: 39, 19: 39, 20: 28, 21: 6, 22: 28, 23: 12, 24: 13, 25: 20, 26: 20, 27: 18, 28: 33, 29: 21, 30: 13, 31: 12, 32: 6, 33: 6, 34: 6, 35: 12, 36: 24, 37: 37, 38: 37, 39: 17, 40: 6, 41: 28, 42: 10, 43: 13, 44: 11, 45: 6, 46: 16, 47: 9, 48: 14, 49: 17, 50: 13, 51: 16, 52: 16, 53: 38, 54: 34, 55: 6, 56: 29, 57: 9, 58: 38, 59: 27, 60: 6, 61: 13, 62: 6, 63: 24, 64: 6, 65: 9, 66: 11, 67: 20, 68: 6, 69: 21, 70: 1, 71: 38, 72: 13, 73: 6, 74: 6, 75: 11, 76: 24, 77: 21, 78: 18, 79: 7, 80: 20, 81: 6, 82: 38, 83: 38, 84: 38, 85: 34, 86: 20, 87: 11, 88: 30, 89: 6, 90: 14, 91: 8, 92: 24, 93: 23, 94: 9, 95: 4, 96: 24, 97: 37, 98: 13, 99: 30, 100: 38, 101: 10, 102: 13, 103: 38, 104: 38, 105: 38, 106: 30, 107: 11, 108: 12, 109: 20, 110: 6, 111: 20, 112: 18, 113: 23, 114: 25, 115: 33, 116: 18, 117: 12, 118: 30, 119: 6, 120: 38, 121: 6, 122: 29, 123: 8, 124: 17, 

Movies that the User has already watched for comparision

In [26]:
df=A_watched.loc[user_id]
result_map = {}

for key,value in df[df.notnull()].items():
    result_map[key] = value

movies=pd.read_csv("movies.csv")
movies=movies.set_index(keys=['MovieID'])
result_map=dict(sorted(result_map.items(), key=lambda item: item[1],reverse=True))

movies_currently_watched=[]
for movieID,rating in result_map.items():
    movie=movies.loc[movieID]
    movies_currently_watched.append([movieID,rating,movie['Title'],movie['Genres']])

print("Movies Watched by User :", user_id)
for movie in movies_currently_watched:  # Display top 10 recommended movies
    print(f"Title : {movie[2]} , MovieID : {movie[0]} , Rating : {movie[1]} , Genres : {movie[3]}")  # Assuming the first column is movie titles

Movies Watched by User : 7000
Title : Big Green, The (1995) , MovieID : 54 , Rating : 5.0 , Genres : Children's|Comedy
Title : Last Detail, The (1973) , MovieID : 3200 , Rating : 5.0 , Genres : Comedy|Drama
Title : Leaving Las Vegas (1995) , MovieID : 25 , Rating : 4.0 , Genres : Drama|Romance
Title : Aliens (1986) , MovieID : 1200 , Rating : 4.0 , Genres : Action|Sci-Fi|Thriller|War
Title : Bio-Dome (1996) , MovieID : 65 , Rating : 3.0 , Genres : Comedy
Title : Heidi Fleiss: Hollywood Madam (1995) , MovieID : 99 , Rating : 2.0 , Genres : Documentary
Title : Free Willy (1993) , MovieID : 455 , Rating : 2.0 , Genres : Adventure|Children's|Drama
Title : Men Cry Bullets (1997) , MovieID : 2980 , Rating : 1.0 , Genres : Drama
